# Linked Brushing Demo

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/movingpandas-examples/main?filepath=tech-demos/linked-brushing.ipynb)

This notebook demonstrates linked brushing with **Holoviews.selection**.


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from datetime import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
import holoviews as hv
import hvplot.pandas  # noqa
from datashader.utils import lnglat_to_meters
from holoviews.element import tiles
from holoviews.util.transform import dim
from holoviews.selection import link_selections
from holoviews.operation import gridmatrix
from holoviews.operation.element import histogram
from holoviews import opts

In [ ]:
hv.__version__

In [ ]:
opts.defaults(opts.Overlay(active_tools=['wheel_zoom']))

In [ ]:
gdf = gpd.read_file('../data/ais.gpkg', rows=1000)

In [ ]:
gdf.head()

## Pandas DataFrame.hvplot

In [ ]:
gdf.loc[:, 'x'], gdf.loc[:, 'y'] = lnglat_to_meters(gdf.geometry.x, gdf.geometry.y)
df = pd.DataFrame(gdf)

In [ ]:
hist_plot = df.where((df.SOG>0) & (df.SOG<50)).hvplot.hist("SOG",  bins=20, width=400, height=300)  

In [ ]:
map_plot = df.hvplot.scatter(x='x', y='y', width=400, height=300)

In [ ]:
link_selections(tiles.CartoLight() * map_plot + hist_plot)

## Geopandas GeoDataFrame.hvplot

In [ ]:
gdf_map = gdf.hvplot(geo=True, tiles='CartoLight', width=400, height=300)

In [ ]:
gdf_hist = pd.DataFrame(gdf).where((gdf.SOG>0) & (gdf.SOG<50)).hvplot.hist("SOG",  bins=20, width=400, height=300)

In [ ]:
link_selections(gdf_map + gdf_hist)

## Datashade

In [ ]:
datashade = df.hvplot.scatter(x='x', y='y', datashade=True, width=400, height=300)

In [ ]:
link_selections(datashade + hist_plot)#.cols(1)

In [ ]:
link_selections(tiles.CartoLight() * datashade + hist_plot)#.cols(1)

## Bar plots (unsupported)

It would be nice to add a bar plot with counts per ship type but bar plots are currently not supported, see http://holoviews.org/user_guide/Linked_Brushing.html

In [ ]:
bar_plot = df.groupby('ShipType').agg({'SOG':'count'}).rename(columns={'SOG':'count'}).hvplot.barh(width=400, height=400)
bar_plot